In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
import seaborn as sn
from math import floor, ceil
from pylab import rcParams
from sklearn import preprocessing
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [ ]:
#Import training data

features = np.loadtxt('vec_mat_features_icex_src_0.01train.csv',delimiter=",")
temp_labels = np.loadtxt('vec_mat_clabels_icex_src_0.01train.csv',delimiter=",")
labels_t = []

X_train = preprocessing.scale(features)

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))
    
labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series): 
    return pd.get_dummies(series.astype(str))

y_train = encode(labels_t)
labels = list(y_train.columns.values)

y_train = pd.DataFrame.as_matrix(y_train)


In [ ]:
f = plt.figure(figsize=(20,10))
#plt.pcolormesh(range(1056), range(4701), X_train, cmap='jet',vmin=-2, vmax=2)
plt.plot(range(1056),X_train[0,:])
plt.title('Training Data')
# set the limits of the plot to the limits of the data
#ax.axis([x.min(), x.max(), y.min(), y.max()])
#plt.colorbar()
plt.show()

In [ ]:
#Import test data

X_test = np.loadtxt('vec_mat_features_icex_src_test6.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_clabels_icex_src_test6.csv',delimiter=",")
y_test= []

X_test = preprocessing.scale(X_test)

for l in range(len(temp_ytest)):
    y_test.append(str(temp_ytest[l]))
    
y_test = np.array(y_test)
y_test = y_test.ravel()
label_test = y_test.ravel()

y_test = encode(y_test)
y_test = pd.DataFrame.as_matrix(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# 1 layer; with dropout; 128 node sigmoid; softmax output; x-entropy loss; adam

drate_l = [0.1,0.2,0.3,0.4,0.5]
n_node_l = [32,64,128,256,512]
batch_size_l = [32,64,128,256,512]
activation_l = [1,2]
lr_l = [0.0001,0.001,0.01,0.1]

for d in range(len(drate_l)):
    for n in range(len(n_node_l)):
        for b in range(len(batch_size_l)):
            for a in range(len(activation_l)):
                for l in range(len(lr_l)):
                    
                    drate = drate_l[d]
                    n_node = n_node_l[n]
                    batch_size = batch_size_l[b]
                    lr = lr_l[l]
                    
                    if a == 0:
                        activation = tf.nn.relu
                        act = 'relu'
                    else:
                        activation = tf.nn.sigmoid
                        act = 'sigmoid'
                            
                    model = keras.Sequential([
                    keras.layers.Dropout(drate),
                    keras.layers.Dense(n_node, activation),
                    keras.layers.Dense(95, tf.nn.softmax)])
                    
                    optimizer = keras.optimizers.Adam(lr)
                    
                    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
                    
                    model.fit(X_train, y_train, batch_size, epochs=5, verbose=0, validation_split=0.0, validation_data=None, shuffle=True)
                    
                    test_loss, test_acc = model.evaluate(X_test,y_test,verbose=0)

                    print('dropout:', drate, '; n_node:', n_node, '; batch size:', batch_size, '; lr:', lr,'; activation:',act, '; Test accuracy:', test_acc)


In [ ]:
#Training

drate = 0.5
n_node = 256
batch_size = 128
activation = tf.nn.sigmoid
#loss='kullback_leibler_divergence'
loss='categorical_crossentropy'

model = keras.Sequential([
    keras.layers.Dense(n_node, activation),
    #keras.layers.Dropout(drate),
    #keras.layers.Dense(n_node, activation),
    #keras.layers.Dropout(drate),
    #keras.layers.Dense(n_node, activation),
    #keras.layers.Dropout(drate),
    #keras.layers.Dense(n_node, activation),
    #keras.layers.Dropout(drate),
    #keras.layers.Dense(n_node, activation),
    keras.layers.Dense(95, tf.nn.softmax)])

lr = 0.01
optimizer = keras.optimizers.Adam(lr)

model.compile(optimizer,loss, metrics=['accuracy'])

t = time.time()
infdb = model.fit(X_train, y_train, batch_size,verbose = True, epochs=23, validation_split=0.0, validation_data=(X_test,y_test), shuffle=True)
elapsed = time.time() - t
print(elapsed)

In [ ]:
#Testing

test_loss, test_acc = model.evaluate(X_test,y_test)
train_loss, train_acc = model.evaluate(X_train,y_train)

print('Test accuracy:', test_acc)
print('Training accuracy:', train_acc)

predictions = model.predict(X_test)

pred_labels = []
for i in np.argmax(predictions, axis=1):
    pred_labels.append(labels[i])
    
    

In [ ]:
epoch = np.arange(1,24)
fig1 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = INF')
plt.plot(epoch, infdb.history['loss'], label = 'Training')
plt.plot(epoch, infdb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(infdb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(infdb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
#fig1.savefig('cfnn_INFdb.png')
'''
fig2 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 10dB')
plt.plot(epoch, tendb.history['loss'], label = 'Training')
plt.plot(epoch, tendb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(tendb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(tendb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig2.savefig('cfnn_10db.png')

fig3 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 5dB')
plt.plot(epoch, fivedb.history['loss'], label = 'Training')
plt.plot(epoch, fivedb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(fivedb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(fivedb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig3.savefig('cfnn_5db.png')

fig4 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = 0dB')
plt.plot(epoch, zerodb.history['loss'], label = 'Training')
plt.plot(epoch, zerodb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(zerodb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(zerodb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig4.savefig('cfnn_0db.png')

fig5 = plt.figure()
epoch = np.arange(1,24)
plt.subplot(1,2,1)
plt.suptitle('SNR = -5dB')
plt.plot(epoch, mfivedb.history['loss'], label = 'Training')
plt.plot(epoch, mfivedb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(mfivedb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(mfivedb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig5.savefig('cfnn_m5db.png')

fig6 = plt.figure()
plt.subplot(1,2,1)
plt.suptitle('SNR = -10dB')
plt.plot(epoch, mtendb.history['loss'], label = 'Training')
plt.plot(epoch, mtendb.history['val_loss'],label = 'Testing')
plt.legend()
plt.grid()
plt.xlabel('Epoch Number')
plt.ylabel('Cross Entropy Loss')
plt.axis([1, 23, 0, 4])
plt.subplot(1,2,2)
plt.plot(epoch, 100.*np.array(mtendb.history['acc']),label = 'Training')
plt.plot(epoch, 100.*np.array(mtendb.history['val_acc']),label = 'Testing')
plt.legend()
plt.xlabel('Epoch Number')
plt.ylabel('Percent Accuracy')
plt.grid()
plt.axis([1, 23, 0, 100])
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig6.savefig('cfnn_m10db.png')
'''

In [ ]:
#sort output for plotting

floaty_test = label_test.astype(np.float)
inds = floaty_test.argsort()
sorted_y_test = label_test[inds]

sorted_y_pred = np.array(pred_labels)[inds]

floatlabels = np.array(labels).astype(np.float)
sortedlabels = np.sort(floatlabels)
sortedclassnames = sortedlabels.astype(np.str)

In [ ]:
# Generate confusion matrix to see where prediction errors occur

cm = confusion_matrix(sorted_y_test, sorted_y_pred, sortedclassnames)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#np.fill_diagonal(cm, 0)

index = np.linspace(3,50,95)
index_str = []
for i in range(len(index)):
    index_str.append(str(index[i]))

df_cm = pd.DataFrame(cm)
fig1 = plt.figure(figsize = (20,14))
sn.heatmap(df_cm,cmap="YlGnBu",xticklabels=sortedclassnames,yticklabels=sortedclassnames,linewidths=.3)
plt.xlabel=('True label'),
plt.ylabel=('Predicted label')
plt.title('SNR = INF')
fig1.savefig('cfnn_cm_infdb.png')